# rollups_blobs_economics

> This notebook explores the economics of rollups and blobs from cryo transactions and blocks dataset.

In [1]:
from ethereum_block_explorer.core import get_txs_blocks_mempool

import polars as pl

In [2]:
txs = pl.scan_parquet("transactions/*.parquet")
blocks = pl.scan_parquet("blocks/*.parquet")
mempool = pl.scan_parquet("data/transaction-data/*.parquet")

In [3]:
txs_blocks_mempool = get_txs_blocks_mempool(txs, blocks, mempool)

Currently I am just sorting sequencer addresses by the `from_address`. More robust would probably to match it to the relevant `to_address`. I'm unsure which way is the most accurate right now.

In [22]:
# https://dune.com/queries/3302607
sequencers_l2: dict[str] = {
    "sequencer_addresses": [
        "0xc1b634853cb333d3ad8663715b08f41a3aec47cc",
        "0x6887246668a3b87f54deb3b94ba47a6f63f32985",
        "0xd19d4b5d358258f05d7b411e21a1460d11b0876f",
        "0x6667961f5e9c98a76a48767522150889703ed77d",
        "0xcf2898225ed05be911d3709d9417e86e0b4cfc8f",
        "0x148ee7daf16574cd020afa34cc658f8f3fbd2800",
        "0x2c169dfe5fbbba12957bdd0ba47d9cedbfe260ca7",
        "0x5050f69a9786f081509234f1a7f4684b5e5b76c9",
    ],
    "sequencer_names": [
        "arbitrum",
        "optimism",
        "linea",
        "mantle",
        "scroll",
        "polygon_zkevm",
        "starknet",
        "base",
    ],
}

sequencer_labels = pl.from_dict(sequencers_l2)

In [23]:
sequencer_labels

sequencer_addresses,sequencer_names
str,str
"""0xc1b634853cb333d3ad8663715b08f41a3aec47cc""","""arbitrum"""
"""0x6887246668a3b87f54deb3b94ba47a6f63f32985""","""optimism"""
"""0xd19d4b5d358258f05d7b411e21a1460d11b0876f""","""linea"""
"""0x6667961f5e9c98a76a48767522150889703ed77d""","""mantle"""
"""0xcf2898225ed05be911d3709d9417e86e0b4cfc8f""","""scroll"""
"""0x148ee7daf16574cd020afa34cc658f8f3fbd2800""","""polygon_zkevm"""
"""0x2c169dfe5fbbba12957bdd0ba47d9cedbfe260ca7""","""starknet"""
"""0x5050f69a9786f081509234f1a7f4684b5e5b76c9""","""base"""


In [24]:
sequencer_txs_blocks = (
    txs_blocks_mempool.filter(
        pl.col("from_address").is_in(sequencer_labels["sequencer_addresses"])
    )
    .collect(streaming=True)
    .join(sequencer_labels, left_on="from_address", right_on="sequencer_addresses")
)

In [29]:
sequencer_txs_blocks.group_by(
    "from_address",
    "sequencer_names",
).agg(
    pl.count(), pl.col("tx_gas_cost").mean()
).sort(by="tx_gas_cost", descending=True)

from_address,sequencer_names,count,tx_gas_cost
str,str,u32,f64
"""0x6667961f5e9c98a76a48767522150889703ed77d""","""mantle""",455,0.2770754713651314
"""0xc1b634853cb333d3ad8663715b08f41a3aec47cc""","""arbitrum""",15981,0.11013049692263244
"""0x6887246668a3b87f54deb3b94ba47a6f63f32985""","""optimism""",4559,0.08331374485589356
"""0xcf2898225ed05be911d3709d9417e86e0b4cfc8f""","""scroll""",5327,0.06420822292767576
"""0x148ee7daf16574cd020afa34cc658f8f3fbd2800""","""polygon_zkevm""",1073,0.04114850410694565
"""0x5050f69a9786f081509234f1a7f4684b5e5b76c9""","""base""",10350,0.023382982340113437
